In [44]:
import os
from params import PARAMS
import pickle
import numpy as np
import json

In [45]:
import pandas as pd

def eval_runs(model_names, dataset_names, random_seeds):
    final_results = []
    for model_name in model_names:
        for dataset_name in dataset_names:
            subject_idc = json.load(open(os.path.join('./dataset/train', dataset_name + '.json'), 'r'))
            accs = []
            precs_macro = []
            precs_weighted = []
            recalls_macro = []
            recalls_weighted = []
            aurocs = []
            n_trainable_params = []

            for random_seed in random_seeds:
                path = os.path.join('output', model_name, dataset_name.rstrip('.pickle').lstrip('./dataset/train/'), str(random_seed), 'results.pickle')
                results = pickle.load(open(path, 'rb'))

                accs.append(results['cr']['accuracy'])
                precs_macro.append(results['cr']['macro avg']['precision'])
                precs_weighted.append(results['cr']['weighted avg']['precision'])
                recalls_macro.append(results['cr']['macro avg']['recall'])
                recalls_weighted.append(results['cr']['weighted avg']['recall'])
                aurocs.append(results['auroc'])
                n_trainable_params.append(results['n_params'])
            result_df = pd.DataFrame({'model_name': [model_name for i in random_seeds], 'dataset_name': [dataset_name for i in random_seeds],'train_idc': [subject_idc['train_idc'] for i in random_seeds], 'test_idc': [subject_idc['test_idc'] for i in random_seeds], 'random_seed': random_seeds, 'accuracy': accs, 'precision_macro': precs_macro, 'precision_weighted': precs_weighted, 'recall_macro': recalls_macro, 'recall_weighted': recalls_weighted, 'AUROC': aurocs, 'n_params': n_trainable_params})
            final_results.append(result_df)
    final_results = pd.concat(final_results).reset_index(drop=True)
    final_results.to_csv('./output/results.csv', index=False)

    std_per_dataset = final_results.groupby(['model_name', 'dataset_name']).std().drop(columns=['random_seed'])
    std_per_dataset = std_per_dataset.rename(columns={'accuracy': 'accuracy_std',
                        'precision_macro': 'precision_macro_std', 
                        'precision_weighted': 'precision_weighted_std', 
                        'recall_macro': 'recall_macro_std', 
                        'recall_weighted': 'recall_weighted_std'})
    std_per_dataset = std_per_dataset.drop(columns=['AUROC', 'n_params'])
    mean_per_dataset = final_results.groupby(['model_name', 'dataset_name']).mean().drop(columns='random_seed')
    results_per_dataset = pd.concat([mean_per_dataset, std_per_dataset], axis=1)
    results_per_dataset.to_csv('./output/results_per_dataset.csv')

    std_per_model = final_results.groupby(['model_name']).std().drop(columns=['random_seed'])
    std_per_model = std_per_model.rename(columns={'accuracy': 'accuracy_std',
                        'precision_macro': 'precision_macro_std', 
                        'precision_weighted': 'precision_weighted_std', 
                        'recall_macro': 'recall_macro_std', 
                        'recall_weighted': 'recall_weighted_std'})
    std_per_model = std_per_model.drop(columns=['AUROC', 'n_params'])
    mean_per_model = final_results.groupby(['model_name']).mean().drop(columns='random_seed')
    results_per_model = pd.concat([mean_per_model, std_per_model], axis=1)
    results_per_model.to_csv('./output/results_per_model.csv')
    print(results_per_model)

In [46]:
model_names = ['imagine_fcn', 'imagine_cnn', 'imagine_rnn', 'imagine_gcn', 'imagine_gcn_auto']
# model_names = ['imagine_gcn_auto']
random_seeds = PARAMS['RANDOM_SEEDS']
print(random_seeds)
dataset_names = [f'cross_subject_data_{i}_5_subjects' for i in range(5)]
final_results = eval_runs(model_names, dataset_names, random_seeds)
# for k, v in final_results.items():
#     print(k)
#     print(v['accuracy'])

[688933, 890203, 517391, 195645, 705223]
                  accuracy  precision_macro  precision_weighted  recall_macro  \
model_name                                                                      
imagine_cnn       0.860959         0.861168            0.861234      0.860960   
imagine_fcn       0.315810         0.342058            0.341978      0.310699   
imagine_gcn       0.402187         0.337877            0.338398      0.397517   
imagine_gcn_auto  0.861212         0.862484            0.862389      0.861079   
imagine_rnn       0.847998         0.848835            0.848759      0.847976   

                  recall_weighted     AUROC  n_params  accuracy_std  \
model_name                                                            
imagine_cnn              0.860959  0.907306  371220.0      0.014189   
imagine_fcn              0.315810  0.540470  354308.0      0.073133   
imagine_gcn              0.402187  0.598348  354308.0      0.213441   
imagine_gcn_auto         0.861212  0

In [49]:
final_results = pd.read_csv('./output/results.csv')

std_per_dataset = final_results.groupby(['model_name', 'dataset_name']).std().drop(columns=['random_seed'])
std_per_dataset = std_per_dataset.rename(columns={'accuracy': 'accuracy_std',
                    'precision_macro': 'precision_macro_std', 
                    'precision_weighted': 'precision_weighted_std', 
                    'recall_macro': 'recall_macro_std', 
                    'recall_weighted': 'recall_weighted_std'})
std_per_dataset = std_per_dataset.drop(columns=['AUROC', 'n_params'])
mean_per_dataset = final_results.groupby(['model_name', 'dataset_name']).mean().drop(columns='random_seed')
results_per_dataset = pd.concat([mean_per_dataset, std_per_dataset], axis=1)
results_per_dataset.reset_index()

,model_name,dataset_name,accuracy,precision_macro,precision_weighted,recall_macro,recall_weighted,AUROC,n_params,accuracy_std,precision_macro_std,precision_weighted_std,recall_macro_std,recall_weighted_std
0,imagine_cnn,cross_subject_data_0_5_subjects,0.878709,0.878947,0.878964,0.878744,0.878709,0.919155,371220.0,0.005443,0.005447,0.005476,0.005460,0.005443
1,imagine_cnn,cross_subject_data_1_5_subjects,0.852038,0.852135,0.852304,0.852007,0.852038,0.901347,371220.0,0.006706,0.006732,0.006564,0.006643,0.006706
2,imagine_cnn,cross_subject_data_2_5_subjects,0.856454,0.856837,0.856927,0.856523,0.856454,0.904334,371220.0,0.023912,0.023881,0.023704,0.023729,0.023912
3,imagine_cnn,cross_subject_data_3_5_subjects,0.858899,0.859062,0.859101,0.858879,0.858899,0.905924,371220.0,0.003710,0.003827,0.003766,0.003682,0.003710
4,imagine_cnn,cross_subject_data_4_5_subjects,0.858696,0.858861,0.858872,0.858650,0.858696,0.905771,371220.0,0.004174,0.004217,0.004218,0.004201,0.004174
5,imagine_fcn,cross_subject_data_0_5_subjects,0.316415,0.377103,0.376108,0.312608,0.316415,0.541756,354308.0,0.084337,0.292128,0.290331,0.085285,0.084337
6,imagine_fcn,cross_subject_data_1_5_subjects,0.336073,0.278176,0.279968,0.332269,0.336073,0.554859,354308.0,0.109684,0.293081,0.293227,0.113141,0.109684
7,imagine_fcn,cross_subject_data_2_5_subjects,0.301562,0.285110,0.284597,0.291325,0.301562,0.527545,354308.0,0.055344,0.301057,0.296345,0.056668,0.055344
8,imagine_fcn,cross_subject_data_3_5_subjects,0.305571,0.418098,0.417143,0.301245,0.305571,0.534167,354308.0,0.054298,0.323681,0.321704,0.054771,0.054298
9,imagine_fcn,cross_subject_data_4_5_subjects,0.319429,0.351805,0.352072,0.316047,0.319429,0.544023,354308.0,0.077595,0.275007,0.273823,0.082346,0.077595


In [50]:
std_per_model = final_results.groupby(['model_name']).std().drop(columns=['random_seed'])
std_per_model = std_per_model.rename(columns={'accuracy': 'accuracy_std',
                    'precision_macro': 'precision_macro_std', 
                    'precision_weighted': 'precision_weighted_std', 
                    'recall_macro': 'recall_macro_std', 
                    'recall_weighted': 'recall_weighted_std'})
std_per_model = std_per_model.drop(columns=['AUROC', 'n_params'])
mean_per_model = final_results.groupby(['model_name']).mean().drop(columns='random_seed')
results_per_model = pd.concat([mean_per_model, std_per_model], axis=1)
results_per_model

,accuracy,precision_macro,precision_weighted,recall_macro,recall_weighted,AUROC,n_params,accuracy_std,precision_macro_std,precision_weighted_std,recall_macro_std,recall_weighted_std
model_name,,,,,,,,,,,,
imagine_cnn,0.860959,0.861168,0.861234,0.860960,0.860959,0.907306,371220.0,0.014189,0.014204,0.014115,0.014145,0.014189
imagine_fcn,0.315810,0.342058,0.341978,0.310699,0.315810,0.540470,354308.0,0.073133,0.276979,0.275117,0.075575,0.073133
imagine_gcn,0.402187,0.337877,0.338398,0.397517,0.402187,0.598348,354308.0,0.213441,0.328949,0.327735,0.216193,0.213441
imagine_gcn_auto,0.861212,0.862484,0.862389,0.861079,0.861212,0.907396,358404.0,0.019300,0.018327,0.018306,0.019357,0.019300
imagine_rnn,0.847998,0.848835,0.848759,0.847976,0.847998,0.898643,357556.0,0.006921,0.006888,0.006835,0.006918,0.006921


In [51]:
path = 'output/imagine_cnn/cross_subject_data_0/195645/results.pickle'
results = pickle.load(open(path, 'rb'))

In [55]:
results['history']['val_acc']

[0.28912869095802307,
 0.2906447947025299,
 0.30393293499946594,
 0.29135823249816895,
 0.28877195715904236,
 0.29626327753067017,
 0.2916257977485657,
 0.294568806886673,
 0.2949255406856537,
 0.30009809136390686,
 0.2949255406856537,
 0.29474717378616333,
 0.2933202385902405,
 0.30214929580688477,
 0.29082316160202026,
 0.28029966354370117,
 0.29296353459358215,
 0.2868099510669708,
 0.29465797543525696,
 0.29421207308769226,
 0.2916257977485657,
 0.2868991196155548,
 0.2931418716907501,
 0.29055559635162354,
 0.2914474308490753,
 0.28377774357795715,
 0.2889503240585327,
 0.29055559635162354,
 0.28841522336006165,
 0.29251760244369507,
 0.29180416464805603,
 0.2860964834690094,
 0.28663158416748047,
 0.2899313271045685,
 0.2874342203140259,
 0.2857397496700287,
 0.29171496629714966,
 0.28288593888282776,
 0.29091233015060425,
 0.28636404871940613,
 0.2914474308490753,
 0.2894854247570038,
 0.2901988625526428,
 0.2857397496700287,
 0.2838669419288635,
 0.28618568181991577,
 0.2915365